In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
dataset = pd.read_csv('dataset_loker_downsynd.csv')

print("Dataset Overview:")
print(dataset.head())

Dataset Overview:
      Nama Pekerjaan                Perusahaan      Lokasi          Gaji  \
0      Asisten Dapur         PT Roti Nusantara     Jakarta  Rp 2.500.000   
1      Petugas Kebun        Taman Kota Mandiri     Bandung  Rp 2.000.000   
2  Operator Komputer           PT Digital Jaya    Surabaya  Rp 3.000.000   
3  Pembersih Ruangan       PT Bersih Sejahtera  Yogyakarta  Rp 2.200.000   
4  Fotografer Pemula  Studio Kreatif Nusantara    Denpasar  Rp 3.500.000   

                                 Deskripsi Pekerjaan       Kategori Minat  \
0  Membantu koki menyiapkan bahan masakan dan men...              Memasak   
1  Merawat tanaman, menyiram bunga, dan menjaga k...             Berkebun   
2  Mengoperasikan komputer untuk tugas-tugas sede...      Teknologi dasar   
3  Membersihkan ruangan kerja dan menjaga kebersi...  Lingkungan dan alam   
4  Mengambil foto sederhana dan membantu pengedit...      Fotografi, Seni   

                                Kemampuan Dibutuhkan  \
0  Men

In [ ]:
dataset['Kategori Minat'] = dataset['Kategori Minat'].fillna('')

dataset['Kategori Minat'] = dataset['Kategori Minat'].str.split(', ')

In [ ]:
minat_encoder = MultiLabelBinarizer()
minat_encoded = minat_encoder.fit_transform(dataset['Kategori Minat'])

In [ ]:
X_train, X_test = train_test_split(minat_encoded, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')  # Output layer
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 167ms/step - accuracy: 0.0109 - loss: 0.6960 - val_accuracy: 0.1111 - val_loss: 0.6731
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1208 - loss: 0.6661 - val_accuracy: 0.2222 - val_loss: 0.6420
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1895 - loss: 0.6353 - val_accuracy: 0.2222 - val_loss: 0.6043
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1999 - loss: 0.5917 - val_accuracy: 0.2778 - val_loss: 0.5599
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3937 - loss: 0.5410 - val_accuracy: 0.2222 - val_loss: 0.5112
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3342 - loss: 0.4893 - val_accuracy: 0.2222 - val_loss: 0.4622
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3638 - loss: 0.4466 - val_accuracy: 0.2222 - val_loss: 0.4177
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3633 - loss: 0.3976 - val_accuracy: 0.2222 - val_loss: 0.3802

In [ ]:
def rekomendasi_jobspark(minat_user, dataset, minat_encoder, model, top_n=5):
    user_minat_vector = minat_encoder.transform([minat_user])

    user_prediction = model.predict(user_minat_vector)

    nilai_similaritas = cosine_similarity(user_prediction, minat_encoded)

    similiaritas_indeks = nilai_similaritas[0].argsort()[-top_n-1:-1][::-1]

    print("Top Recommendations:")
    for i in similiaritas_indeks:
        rekomendasi_jobspark = dataset.iloc[i]
        print(f"Nama Pekerjaan: {rekomendasi_jobspark['Nama Pekerjaan']}")
        print(f"Perusahaan: {rekomendasi_jobspark['Perusahaan']}")
        print(f"Lokasi: {rekomendasi_jobspark['Lokasi']}")
        print(f"Kualifikasi: {rekomendasi_jobspark['Kualifikasi']}")
        print(f"Minimal Pengalaman: {rekomendasi_jobspark['Minimal Pengalaman']}")
        print(f"Kategori Minat: {rekomendasi_jobspark['Kategori Minat']}")
        print(f"Kemampuan Dibutuhkan: {rekomendasi_jobspark['Kemampuan Dibutuhkan']}")
        print(f"Kondisi Kesehatan: {rekomendasi_jobspark['Kondisi Kesehatan']}")
        print("-" * 40)

In [ ]:
minat_user = ['Musik']
rekomendasi_jobspark(minat_user, dataset, minat_encoder, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Top Recommendations:
Nama Pekerjaan: Asisten Pelatih Musik
Perusahaan: Sanggar Musik Nusantara
Lokasi: Surabaya
Kualifikasi: Lulusan SMA/SMK
Minimal Pengalaman: 0-1 tahun
Kategori Minat: ['Musik', 'Seni']
Kemampuan Dibutuhkan: Kesabaran, Kreativitas, Mendengarkan instruksi
Kondisi Kesehatan: Membutuhkan lingkungan kerja tenang
----------------------------------------
Nama Pekerjaan: Teknisi Alat Musik
Perusahaan: Toko Musik Harmoni
Lokasi: Medan
Kualifikasi: Lulusan SMA/SMK
Minimal Pengalaman: 0-2 tahun
Kategori Minat: ['Musik', 'Seni']
Kemampuan Dibutuhkan: Pemecahan masalah sederhana, Kreativitas, Ketelitian
Kondisi Kesehatan: Membutuhkan lingkungan kerja tenang
----------------------------------------
Nama Pekerjaan: Teknisi Headphone
Perusahaan: Toko Audio Digital
Lokasi: Bandung
Kualifikasi: Lulusan SMA/SMK
Minimal Pengalaman: 1-2 tahun
Kategori Minat: ['Teknologi dasar', 'Musik']
Kemampuan Dibutuhkan: Pemecahan masalah sederhana, Kreativitas,